# Final project - Hyper parameter optimization

## Installing packages (if running on colab)

**IMPORTANT**: Uncomment and run the cell below and then restart the Runtime (Menu Runtime > Restart Runtime, or with Ctrl + M .), then run it again. If you do not do that, then you will get errors. You only need to run it again if your Google Colab / Kaggle instance is restarted or lost.

In [ ]:
'''
! pip install --upgrade scipy
! pip install --upgrade pandas
! pip install ipywidgets
! pip uninstall -y pykeen
! pip install git+https://github.com/pykeen/pykeen.git@v1.5.0
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git@v1.5.0
from pkg_resources import require
require('pykeen')
'''

After you install the packages above, you can just run from this cell onwards (Ctrl + F10 when this is selected)

If SAVE_TO_DRIVE is set to True, the following cell enables storing of hpo results to your google drive account. Authentication required, and only works on colab instances (as far as we know)

In [1]:
SAVE_TO_DRIVE = False
if (SAVE_TO_DRIVE):
    from google.colab import drive
    drive.mount('/content/drive')


## Imports and parameters

In [2]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import pandas as pd
import torch
import gc
from pykeen.hpo import hpo_pipeline

How many epochs to use on training

In [3]:
N_EPOCHS = 100

How many seconds to use for each hyperparameter optimization trial. The optimizer will run as many trials as possible and stop after finishing the one that was runningwhen time ran out.

In [4]:
TIME_PER_TRIAL = 900

Always run on GPU

In [5]:
CPU_DEV = 'gpu'

In [6]:

# Use this value for BASE_DATA_URL if working with local data
BASE_DATA_URL = './data'

# Use this value for BASE_DATA_URL if working with data from the github repo
#BASE_DATA_URL = 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/data'

## Retrieving data

The data sets used were already pre stratified into training, testing and validation sets. Using the methods described in the data manipulation notebook they were split into symmetrical and asymmetrical subsets. The results were stored in a github repository for easy access in the further work.

To retrieve the data we first populate a dictionary with the URL of the sets.

In [7]:
dataset_urls = dict()

## wn18rr datasets

In [21]:
dataset_urls['wn18rr-full'] = {
    'train': '/wn18rr/train_wn18rr.txt',
    'validate': '/wn18rr/valid_wn18rr.txt',
    'test': '/wn18rr/test_wn18rr.txt'
    }

In [22]:
dataset_urls['wn18rr-sym'] = {
    'train': '/wn18rr/sym_train_wn18rr.txt',
    'validate': '/wn18rr/sym_valid_wn18rr.txt',
    'test': '/wn18rr/sym_test_wn18rr.txt'
    }


In [23]:
dataset_urls['wn18rr-asym'] = {
    'train': '/wn18rr/asym_train_wn18rr.txt',
    'validate': '/wn18rr/asym_valid_wn18rr.txt',
    'test': '/wn18rr/asym_test_wn18rr.txt'
    }

## fb15k-237 datasets

In [24]:
dataset_urls['fb15k-237-full'] = {
    'train':  '/fb15k-237/train_fb15k-237.txt',
    'validate': '/fb15k-237/valid_fb15k-237.txt',
    'test': '/fb15k-237/test_fb15k-237.txt'
    }

In [25]:
dataset_urls['fb15k-237-sym'] = {
    'train': '/fb15k-237/sym_train_fb15k-237.txt',
    'validate': '/fb15k-237/sym_valid_fb15k-237.txt',
    'test': '/fb15k-237/sym_test_fb15k-237.txt'
    }

In [26]:
dataset_urls['fb15k-237-asym'] = {
    'train': '/fb15k-237/asym_train_fb15k-237.txt',
    'validate': '/fb15k-237/asym_valid_fb15k-237.txt',
    'test': '/fb15k-237/asym_test_fb15k-237.txt'
    }

## Read data from files and create TriplesFactories

In [27]:
datasets = dict()

for key in dataset_urls.keys():
    print(f'Processing: {key}')
    
    datasets[key] = dict()
    
    df_train = pd.read_csv(BASE_DATA_URL + dataset_urls[key]['train'], header=None, sep='\t', names=['head', 'relation','tail'])
    df_validate = pd.read_csv(BASE_DATA_URL + dataset_urls[key]['validate'], header=None, sep='\t', names=['head', 'relation','tail'])
    df_test = pd.read_csv(BASE_DATA_URL + dataset_urls[key]['test'], header=None, sep='\t', names=['head', 'relation','tail'])
    
    datasets[key]['train'] = TriplesFactory.from_labeled_triples(df_train.astype('str').to_numpy())
    entity_mapping = datasets[key]['train'].entity_to_id
    relation_mapping = datasets[key]['train'].relation_to_id
    
    datasets[key]['validate'] = TriplesFactory.from_labeled_triples(df_validate.astype('str').to_numpy(),
                                                                    entity_to_id=entity_mapping,
                                                                    relation_to_id=relation_mapping
                                                                    )
    
    datasets[key]['test'] = TriplesFactory.from_labeled_triples(df_test.astype('str').to_numpy(),
                                                                    entity_to_id=entity_mapping,
                                                                    relation_to_id=relation_mapping
                                                                    )
    print('\n')

Processing: fb15k-237-full




Processing: fb15k-237-sym




Processing: fb15k-237-asym




Processing: wn18rr-full




Processing: wn18rr-sym




Processing: wn18rr-asym


## Models

#### Defining models and their parameters
If doing a partial run, comment out models that should not be included

In [18]:
models = [
          #('TransE', dict(scoring_fct_norm=2)),
          #('TransH', dict()),
          #('TransD', dict()),
          #('TransR', dict()),
          ('RESCAL', dict()),
          ('ComplEx', dict()),
          #('RotatE', dict())
         ]



Define parameters that are specific to a combination of dataset and model.

In [28]:
class md_param:
    def __init__(self, loss, training_loop):
        self.loss = loss
        self.training_loop = training_loop        


model_dataset_param = {('TransE',  'wn18rr'):     md_param('BCEWithLogitsLoss', 'lcwa'),
                       ('TransH',  'wn18rr'):     md_param('MarginRankingLoss', 'slcwa'),
                       ('TransD',  'wn18rr'):     md_param('MarginRankingLoss', 'slcwa'),
                       ('TransR',  'wn18rr'):     md_param('MarginRankingLoss', 'slcwa'),
                       ('RESCAL',  'wn18rr'):     md_param('MarginRankingLoss', 'slcwa'),
                       #('RESCAL',  'wn18rr'):     md_param('CrossEntropyLoss', 'lcwa'),                       
                       ('ComplEx', 'wn18rr'):     md_param('MarginRankingLoss', 'slcwa'),
                       #('ComplEx', 'wn18rr'):     md_param('CrossEntropyLoss', 'lcwa'),
                       ('RotatE',  'wn18rr'):     md_param('BCEWithLogitsLoss', 'lcwa'),

                       ('TransE',  'fb15k-237'):  md_param('MarginRankingLoss', 'slcwa'),
                       ('TransH',  'fb15k-237'):  md_param('MarginRankingLoss', 'slcwa'),
                       ('TransD',  'fb15k-237'):  md_param('MarginRankingLoss', 'slcwa'),
                       ('TransR',  'fb15k-237'):  md_param('CrossEntropyLoss', 'lcwa'),                       
                       ('RESCAL',  'fb15k-237'):  md_param('MarginRankingLoss', 'slcwa'), 
                       #('RESCAL',  'fb15k-237'):  md_param('CrossEntropyLoss', 'lcwa'), 
                       ('ComplEx', 'fb15k-237'):  md_param('MarginRankingLoss', 'slcwa'),
                       #('ComplEx', 'fb15k-237'):  md_param('CrossEntropyLoss', 'lcwa'),
                       ('RotatE',  'fb15k-237'):  md_param('NSSALoss', 'lcwa'),
                      }

In [29]:
def run_hpo_pipeline(dataset_name, dataset, model):
  model_name = model[0]
  model_params = model[1]
  dataset_base = str.join('-',str.split(dataset_name, '-')[:-1])
  print(f"Dataset: {dataset_name}, Model: {model_name}")
  result = hpo_pipeline(  
    n_trials=10,
    #timeout=TIME_PER_TRIAL,
    training=dataset['train'],
    testing=dataset['test'],
    validation=dataset['validate'],
    model=model_name,
    model_kwargs=model_params,
    # set the parameters specific to this combination of dataset and model
    loss=model_dataset_param[model_name, dataset_base].loss,
    training_loop=model_dataset_param[model_name, dataset_base].training_loop,
    training_kwargs=dict(num_epochs=N_EPOCHS,
                         use_tqdm_batch=False),
    training_kwargs_ranges=dict(
                            batch_size=dict(type=int, low=256, high=1024, q=256),                    
                            ),
    device=CPU_DEV,
    metric='MEAN_RECIPROCAL_RANK',
    direction='maximize',
    stopper='early',
    stopper_kwargs=dict(frequency=10, patience=2, relative_delta=0.01),
    model_kwargs_ranges=dict(
                        embedding_dim=dict(type=int, low=32, high=400, q=64)
                        )
    ) 

  return(result)


In [ ]:
for dataset_name, dataset in datasets.items():
  print(dataset_name)

  for model in models:
    gc.collect()
    torch.cuda.empty_cache()
    result = run_hpo_pipeline(dataset_name, dataset, model)
    result.save_to_directory(f'{dataset_name}/{model[0]}')
    if (SAVE_TO_DRIVE):
        result.save_to_directory(f'/content/drive/MyDrive/hpo_results/{dataset_name}/{model[0]}')


[I 2021-11-10 23:28:05,770] A new study created in memory with name: no-name-cc451299-a6be-4a6f-8c4d-71a84fffe264
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.complex.ComplEx'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using regularizer: <class 'pykeen.regularizers.LpRegularizer'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize MEAN_RECIPROCAL_RANK
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
C:\Appl\Anaconda3\envs\cuda\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [32, 400] and step

fb15k-237-full
Dataset: fb15k-237-full, Model: ComplEx


Training epochs on cuda:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=256.
INFO:pykeen.evaluation.evaluator:Evaluation took 56.54s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
INFO:pykeen.evaluation.evaluator:Evaluation took 55.54s seconds
INFO:pykeen.evaluation.evaluator:Evaluation took 55.61s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early after 3 evaluations at epoch 30. The best result hits_at_k=0.009956635855300697 occurred at epoch 10.
INFO:pykeen.training.training_loop:=> loading checkpoint 'C:\Users\hvags\AppData\Local\Temp\tmpkvdk5xf2'
INFO:pykeen.training.training_loop:=> loaded checkpoint 'C:\Users\hvags\AppData\Local\Temp\tmpkvdk5xf2' stopped after having finished epoch 10


Evaluating on cuda:   0%|          | 0.00/17.5k [00:00<?, ?triple/s]

...

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=256.
INFO:pykeen.evaluation.evaluator:Evaluation took 54.13s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
INFO:pykeen.evaluation.evaluator:Evaluation took 53.94s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 20.
INFO:pykeen.evaluation.evaluator:Evaluation took 54.17s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 30.
INFO:pykeen.evaluation.evaluator:Evaluation took 53.95s seconds
INFO:pykeen.evaluation.evaluator:Evaluation took 56.34s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early after 5 evaluations at epoch 50. The best result hits_at_k=0.00047058823529411766 occurred at epoch 30.
INFO:pykeen.training.training_loop:=> loading checkpoint 'C:\Users\hvags\AppData\Local\Temp\tmp0f2ot

Evaluating on cuda:   0%|          | 0.00/17.0k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 58.89s seconds
[I 2021-11-11 09:43:45,997] Trial 6 finished with value: 0.0005250615096142644 and parameters: {'model.embedding_dim': 32, 'loss.margin': 2, 'loss.margin_activation': 'softplus', 'regularizer.weight': 0.8041353218290377, 'optimizer.lr': 0.021198415730593334, 'negative_sampler.num_negs_per_pos': 4, 'training.batch_size': 768}. Best is trial 2 with value: 0.015876730003865875.
C:\Appl\Anaconda3\envs\cuda\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [32, 400] and step=64, but the range is not divisible by `step`. It will be replaced by [32, 352].
  warnings.warn(


Training epochs on cuda:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=32.
INFO:pykeen.evaluation.evaluator:Evaluation took 269.81s seconds
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
